В данном файле проводится тестирования класса "gp_tree_design_tree", в котором реализовано вычисление дерева принятия решения

In [1]:
from gp_tree_design_tree import gp_tree_design_tree
from gp_list_design_tree import *

Проверяем создание дерева. Ожидается, что дерево будте создано и структура дерева выведена в строку. 
Метод роста полный

In [2]:
list_F=[]
list_F.append(list_less(name_feature='x1' ))


list_T=[]
list_T.append(list_nom_class(value=1))
list_T.append(list_nom_class(value=2))
list_T.append(list_nom_class(value=3))
list_T.append(list_nom_class(value=4))
list_T.append(list_nom_class(value=5))
list_T.append(list_nom_class(value=6))
list_T.append(list_nom_class(value=7))
list_T.append(list_nom_class(value=8))

tree=gp_tree_design_tree(list_T=list_T, list_F=list_F, level=0, nom_list='1', type_ini='full',
                limit_level=2)
str_tree=tree.print_tree()
print('\n')
print(str_tree)



less[x1,{'p': 0}](less[x1,{'p': 0}](nom_class:7, nom_class:3), less[x1,{'p': 0}](nom_class:2, nom_class:3))


Тестируем установку коэффициентов. Ожидается, что коэффициенты установятся

In [3]:
koef=tree.get_koef()
print(koef)
i=1
for k in koef.keys():
    koef[k]=i
    i=i+1
print(tree.set_koef(koef))
koef1=tree.get_koef()
print(koef1)
print(tree.print_tree())


{'1_p': 0, '1.1_p': 0, '1.2_p': 0}
True
{'1_p': 1, '1.1_p': 2, '1.2_p': 3}
less[x1,{'p': 1}](less[x1,{'p': 2}](nom_class:7, nom_class:3), less[x1,{'p': 3}](nom_class:2, nom_class:3))


Тестируем вычисление дерева для классификации. В качестве пороговых значений устанавливаем 1, 2, 3. Ожидается, что класс узла 1.1.1 установится в первой строке. Класс узла 1.1.2 вообще не установится. Класс узла 1.2.1 установтися в строки 2,3. 
Класс узла 1.2.2 во все остальные. 

In [4]:
y=pd.Series(np.arange(0,1, 0.1))
df=y.copy()
df=df*10
df.name='x1'
df=df.reset_index()

rez=tree.eval(params={'X':df,'y':y}  )

print(rez)
print(df)


0    7.0
1    2.0
2    2.0
3    3.0
4    3.0
5    3.0
6    3.0
7    3.0
8    3.0
9    3.0
dtype: float64
   index   x1
0      0  0.0
1      1  1.0
2      2  2.0
3      3  3.0
4      4  4.0
5      5  5.0
6      6  6.0
7      7  7.0
8      8  8.0
9      9  9.0


Исследуем работу метода predict как надстройку над методом eval


In [5]:
rez=tree.predict(X=df)
print(rez)

0    7.0
1    2.0
2    2.0
3    3.0
4    3.0
5    3.0
6    3.0
7    3.0
8    3.0
9    3.0
dtype: float64


Исследуем работу метода score для классификационного дерева
создаем пробный выход для бинарной классификации и считаем точность


In [6]:
list_F=[]
list_F.append(list_less(name_feature='x1' ))


list_T=[]
list_T.append(list_nom_class(value=0))
list_T.append(list_nom_class(value=1))
tree=gp_tree_design_tree(list_T=list_T, list_F=list_F, level=0, nom_list='1', type_ini='full',
                limit_level=2)
koef=tree.get_koef()
i=1
for k in koef.keys():
    koef[k]=i
    i=i+1
print(tree.set_koef(koef))
print(tree.print_tree())
str_tree=tree.print_tree()
print('\n')
print(str_tree)

df=pd.Series(np.arange(0,1, 0.1))
df=df*10
df.name='x1'
df=df.reset_index()

y=np.random.randint(0,2, len(df))
y=pd.Series(y, index=df.index)
print(df)
print(y)

y_pred=tree.predict(X=df)
print('Сравниваем "реальны" и вычисленный выходы')
print(pd.concat([y,y_pred], axis=1))
f1_score=tree.score(X=df, y=y)
print(f1_score)

True
less[x1,{'p': 1}](less[x1,{'p': 2}](nom_class:0, nom_class:0), less[x1,{'p': 3}](nom_class:0, nom_class:0))


less[x1,{'p': 1}](less[x1,{'p': 2}](nom_class:0, nom_class:0), less[x1,{'p': 3}](nom_class:0, nom_class:0))
   index   x1
0      0  0.0
1      1  1.0
2      2  2.0
3      3  3.0
4      4  4.0
5      5  5.0
6      6  6.0
7      7  7.0
8      8  8.0
9      9  9.0
0    0
1    1
2    0
3    1
4    0
5    0
6    1
7    0
8    1
9    1
dtype: int32
Сравниваем "реальны" и вычисленный выходы
   0    1
0  0  0.0
1  1  0.0
2  0  0.0
3  1  0.0
4  0  0.0
5  0  0.0
6  1  0.0
7  0  0.0
8  1  0.0
9  1  0.0
0.3333333333333333


проверяем многомерный случай. Когда на вход будет многомерным

In [7]:
list_F=[]
list_F.append(list_less(name_feature='x1' ))
list_F.append(list_less(name_feature='x2' ))

list_T=[]
list_T.append(list_nom_class(value=0))
list_T.append(list_nom_class(value=1))
tree=gp_tree_design_tree(list_T=list_T, list_F=list_F, level=0, nom_list='1', type_ini='full',
                limit_level=2)
koef=tree.get_koef()

for k in koef.keys():
    koef[k]=np.around(np.random.rand(),1)
    
print(tree.set_koef(koef))
print(tree.print_tree())


# df=np.random.rand(10,2)
df=np.arange(0,1,0.1)
df=df[:, np.newaxis]
df=np.concatenate([df,df], axis=1)
df=pd.DataFrame(df, columns=['x1', 'x2'])

df['y']=np.random.randint(0,2, len(df))


y_pred=tree.predict(X=df)
y_pred.name='y_pred'
print('Сравниваем "реальны" и вычисленный выходы')

print(pd.concat([df,y_pred], axis=1))
f1_score=tree.score(X=df, y=df['y'])
print(f1_score)

True
less[x2,{'p': 1.0}](less[x1,{'p': 0.8}](nom_class:0, nom_class:1), less[x2,{'p': 0.4}](nom_class:0, nom_class:0))
Сравниваем "реальны" и вычисленный выходы
    x1   x2  y  y_pred
0  0.0  0.0  1     0.0
1  0.1  0.1  0     0.0
2  0.2  0.2  1     0.0
3  0.3  0.3  0     0.0
4  0.4  0.4  0     0.0
5  0.5  0.5  1     0.0
6  0.6  0.6  0     0.0
7  0.7  0.7  0     0.0
8  0.8  0.8  0     1.0
9  0.9  0.9  1     1.0
0.5238095238095238


In [8]:
# Проверяем процедуру подбора пороговых значений путем полного перебора
df=np.random.rand(10,2)
# df=np.arange(0,1,0.1)
# df=df[:, np.newaxis]
# df=np.concatenate([df,df], axis=1)
df=pd.DataFrame(df, columns=['x1', 'x2'])

df['y']=np.random.randint(0,2, len(df))

flag=0
for i in range(10):
    for j in range(10):
        for k in range(10):
            koef_list=[i/10., j/10., k/10.]
            n=0
            for key in koef.keys():
                koef[key]=koef_list[n]
                n+=1
                tree.set_koef(koef)
                f1_score=tree.score(X=df, y=df['y'])
                if flag==0:
                    flag=1
                    best_koef=koef.copy()
                    best_f1_score=f1_score

                elif f1_score> best_f1_score:
                    best_koef=koef.copy()
                    best_f1_score=f1_score

tree.set_koef(best_koef)

y_pred=tree.predict(X=df)
y_pred.name='y_pred'
print('Сравниваем "реальны" и вычисленный выходы')

print(pd.concat([df,y_pred], axis=1))
f1_score=tree.score(X=df, y=df['y'])
print()
print(f1_score)
print(tree.print_tree())                

Сравниваем "реальны" и вычисленный выходы
         x1        x2  y  y_pred
0  0.094652  0.897487  1     0.0
1  0.696088  0.219408  1     1.0
2  0.474791  0.844404  1     0.0
3  0.114632  0.498890  0     0.0
4  0.071347  0.107344  0     0.0
5  0.761530  0.140171  1     1.0
6  0.977320  0.898567  0     0.0
7  0.782458  0.359745  0     0.0
8  0.822156  0.784521  0     0.0
9  0.692789  0.822938  1     0.0

0.6703296703296704
less[x2,{'p': 0.3}](less[x1,{'p': 0.1}](nom_class:0, nom_class:1), less[x2,{'p': 0.9}](nom_class:0, nom_class:0))


Тестируем функцию fit

In [9]:
# Проверяем процедуру подбора пороговых значений путем полного перебора
df=np.random.rand(10,2)

df=pd.DataFrame(df, columns=['x1', 'x2'])

df['y']=np.random.randint(0,2, len(df))

e=tree.fit(X=df,y=df['y'])
print(f'loss: {e}')
y_pred=tree.predict(X=df)
y_pred.name='y_pred'
print('Сравниваем "реальны" и вычисленный выходы')

print(pd.concat([df,y_pred], axis=1))
f1_score=tree.score(X=df, y=df['y'])
print()
print(f1_score)
print(tree.print_tree())                

Optimization terminated successfully.
         Current function value: 0.588235
         Iterations: 9
         Function evaluations: 44
loss: 0.5882352941176471
Сравниваем "реальны" и вычисленный выходы
         x1        x2  y  y_pred
0  0.095146  0.134564  1     0.0
1  0.653868  0.322747  0     0.0
2  0.978359  0.715279  1     0.0
3  0.819519  0.064979  0     0.0
4  0.828792  0.469557  0     0.0
5  0.680094  0.708905  0     0.0
6  0.603327  0.192031  0     0.0
7  0.325164  0.170126  0     0.0
8  0.062096  0.649114  0     0.0
9  0.670621  0.297425  1     0.0

0.4117647058823529
less[x2,{'p': 0.05639814679941024}](less[x1,{'p': 0.42415675367265826}](nom_class:0, nom_class:1), less[x2,{'p': 0.4811889213899213}](nom_class:0, nom_class:0))


d:\user\Projects\Python\GP_design_tree\gp_tree_design_tree.py:190: OptimizeWarning: Unknown solver options: gtol
  res = minimize(self.loss, x0, method=method, options={'gtol': 1e-6, 'disp': True},args=args)


Тестируем оптимизацию через DE

In [10]:
list_F=[]
list_F.append(list_less(name_feature='x1' ))
list_F.append(list_less(name_feature='x2' ))
list_T=[]
list_T.append(list_nom_class(value=0))
list_T.append(list_nom_class(value=1))

tree=gp_tree_design_tree(list_T=list_T, list_F=list_F, level=0, nom_list='1', type_ini='full',
                limit_level=2)
str_tree=tree.print_tree()

# Проверяем функцию fit
df=np.random.rand(10,2)
df=pd.DataFrame(df, columns=['x1', 'x2'])
df['y']=np.random.randint(0,2, len(df))

e=tree.fit(X=df,y=df['y'], method='DE')
print(f'loss: {e}')
y_pred=tree.predict(X=df)
y_pred.name='y_pred'
print('Сравниваем "реальны" и вычисленный выходы')

print(pd.concat([df,y_pred], axis=1))
f1_score=tree.score(X=df, y=df['y'])
print()
print(f1_score)
print(tree.print_tree())   

start evolution
iteration 0: min=0.4, mean=0.6503298368298368, max=0.8333333333333334
iteration 1: min=0.3030303030303032, mean=0.6017206682206683, max=0.8333333333333334
iteration 2: min=0.3030303030303032, mean=0.5525281385281385, max=0.7142857142857143
iteration 3: min=0.3030303030303032, mean=0.5188737373737374, max=0.7142857142857143
iteration 4: min=0.3030303030303032, mean=0.48443722943722956, max=0.7142857142857143
iteration 5: min=0.3030303030303032, mean=0.46321142746142746, max=0.7142857142857143
iteration 6: min=0.19999999999999984, mean=0.4360982905982907, max=0.6666666666666667
iteration 7: min=0.19999999999999984, mean=0.3998165168165169, max=0.5494505494505494
iteration 8: min=0.19999999999999984, mean=0.3773989898989899, max=0.5494505494505494
iteration 9: min=0.19999999999999984, mean=0.3565394605394607, max=0.5494505494505494
iteration 10: min=0.19999999999999984, mean=0.3395397935397937, max=0.5494505494505494
iteration 11: min=0.19999999999999984, mean=0.3313028083

In [19]:
list_F=[]

list_F.append(list_less(name_feature='x1' ))
list_F.append(list_less(name_feature='x2' ))
list_T=[]
list_T.append(list_regr_const())

tree=gp_tree_design_tree(list_T=list_T, list_F=list_F, level=0, nom_list='1', type_ini='full',
                limit_level=4)
str_tree=tree.print_tree()

# Проверяем функцию fit
df=np.random.rand(10,2)
df=pd.DataFrame(df, columns=['x1', 'x2'])
df['y']=2*df['x1']-3*df['x2']

e=tree.fit(X=df,y=df['y'], method='DE',metric='mse')
print(f'loss: {e}')
y_pred=tree.predict(X=df)
y_pred.name='y_pred'
print('Сравниваем "реальны" и вычисленный выходы')

print(pd.concat([df,y_pred], axis=1))
mse_score=tree.score(X=df, y=df['y'], metric='mse')
print()
print(mse_score)
print(tree.print_tree())    

start evolution
iteration 0: min=1.249815764916597, mean=1.9380545323061835, max=2.7357389407327894
iteration 1: min=0.9486048974082049, mean=1.7361886384440282, max=2.7357389407327894
iteration 2: min=0.6713504615676427, mean=1.5509644753884841, max=2.5751823190061454
iteration 3: min=0.6713504615676427, mean=1.407471536914985, max=2.2463047394343736
iteration 4: min=0.6713504615676427, mean=1.3305671768188356, max=1.888160837932686
iteration 5: min=0.44057634694513476, mean=1.2304720990816385, max=1.8638011405295962
iteration 6: min=0.44057634694513476, mean=1.1459213040481857, max=1.784735952563223
iteration 7: min=0.3709706164270391, mean=1.035253316806697, max=1.746166180490351
iteration 8: min=0.3709706164270391, mean=0.9579806550137381, max=1.746166180490351
iteration 9: min=0.3709706164270391, mean=0.8943347395277715, max=1.654214799882195
iteration 10: min=0.3709706164270391, mean=0.8355897823104699, max=1.654214799882195
iteration 11: min=0.34183603813292873, mean=0.801222671

In [21]:
list_F=[]

list_F.append(list_less(name_feature='x1' ))
list_F.append(list_less(name_feature='x2' ))
list_T=[]
list_T.append(list_regr_const())

tree=gp_tree_design_tree(list_T=list_T, list_F=list_F, level=0, nom_list='1', type_ini='full',
                limit_level=2)
str_tree=tree.print_tree()

# Проверяем функцию fit
df=np.random.rand(10,2)
df=pd.DataFrame(df, columns=['x1', 'x2'])
df['y']=2*df['x1']+3*df['x2']

e=tree.fit(X=df,y=df['y'], method='Nelder-Mead',metric='mse')
print(f'loss: {e}')
y_pred=tree.predict(X=df)
y_pred.name='y_pred'
print('Сравниваем "реальны" и вычисленный выходы')

print(pd.concat([df,y_pred], axis=1))
mse_score=tree.score(X=df, y=df['y'], metric='mse')
print()
print(mse_score)
print(tree.print_tree())    

d:\user\Projects\Python\GP_design_tree\gp_tree_design_tree.py:190: OptimizeWarning: Unknown solver options: gtol
  res = minimize(self.loss, x0, method=method, options={'gtol': 1e-6, 'disp': True},args=args)


Optimization terminated successfully.
         Current function value: 1.559704
         Iterations: 150
         Function evaluations: 271
loss: 1.5597036850292412
Сравниваем "реальны" и вычисленный выходы
         x1        x2         y    y_pred
0  0.743652  0.179339  2.025321  2.498093
1  0.964348  0.908331  4.653688  2.498093
2  0.393065  0.982025  3.732206  2.498093
3  0.000630  0.602301  1.808163  2.498093
4  0.406712  0.443955  2.145289  2.498093
5  0.088062  0.163540  0.666744  2.498093
6  0.592034  0.468394  2.589251  2.498093
7  0.535429  0.007835  1.094363  2.498093
8  0.691122  0.953601  4.243046  2.498093
9  0.408481  0.401929  2.022748  2.498093

1.5597036850292412
less[x1,{'p': -1.9478739006396422}](less[x2,{'p': -0.8060307091676999}](const{'const': 1.4116876226296773}, const{'const': 5.864020660628974}), less[x2,{'p': -3.0129553016774016}](const{'const': 1.464729693475335}, const{'const': 2.498093405030615}))
